In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from MOC import calculate_MOC, AMOC_max, approx_lats
from grid import create_tdepth, find_array_idx
from tqdm import tqdm_notebook
from regions import Atlantic_mask, boolean_mask
from paths import file_ex_ocn_ctrl, path_results, CESM_filename, file_ex_ocn_lpd, path_data
from plotting import discrete_cmap, shifted_color_map
from timeseries import IterateOutputCESM, lowpass
from xr_DataArrays import xr_DZ, xr_DXU
from xr_regression import xr_lintrend, xr_linear_trend

# AMOC plot

In [ ]:
AMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_ctrl.nc', decode_times=False)
AMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_rcp.nc' , decode_times=False)
AMOC_lpd  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lpd.nc' , decode_times=False)
AMOC_lr1  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lr1.nc' , decode_times=False)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(6.4,3), sharey=True, constrained_layout=True)
ax[0].set_ylabel('AMOC @ 26N, 1000m')
for i, AMOC_max in enumerate([AMOC_ctrl, AMOC_rcp]):
    c = ['C0', 'C1', 'C3'][i]
    ax[0].plot(AMOC_max.time/365-[0,1800,1800][i], AMOC_max, c=c, alpha=.3, lw=.5)
    ax[0].plot(AMOC_max.time[60:-60]/365-[0,1800,1800][i], lowpass(AMOC_max,120)[60:-60],
               c=c, label=['HIGH CTRL','HIGH RCP'][i])
    if i==1:
        ax[0].plot(AMOC_max.time/365-1800, xr_lintrend(AMOC_max)+2, c=c, ls='--')
ax[0].text(240, 6, f'{xr_linear_trend(AMOC_rcp).values*100*365:3.2f}\nSv/100yr', color='C1')

# RAPID_time = 310+np.arange(0,len(RAPID.time))/12/60
# ax[0].plot(RAPID_time, RAPID['moc_mar_hc10'].rolling(time=60, center=True).mean(), c='C6', alpha=.3)
# ax[0].plot(RAPID_time, RAPID['moc_mar_hc10'].rolling(time=60*12*10, center=True).mean(), c='C6', label='RAPID')
    
for i, AMOC_max in enumerate([AMOC_lpd, AMOC_lr1]):
    ax[1].plot(AMOC_max.time/365-[0,1500,800,1500][i], AMOC_max, c=f'C{i}', alpha=.3, lw=.5)
    ax[1].plot(AMOC_max.time[60:-60]/365-[0,1500,800,1500][i], lowpass(AMOC_max,120)[60:-60],
               c=f'C{i}', label=['LOW CTRL','LOW RCP'][i])
    if i in [1,2]:
        ax[1].plot(AMOC_max.time/365-[0,1500,800,1500][i], xr_lintrend(AMOC_max)+2, c=f'C{i}', ls='--')
        ax[1].text([0,540,1100][i], 6, f'{xr_linear_trend(AMOC_max).values*100*365:3.2f}\nSv/100yr', color=f'C{i}')
        
for i in range(2):
    ax[i].set_ylim((0,29.5))
    ax[i].set_title(['HIGH CESM1.04', 'LOW CESM1.12'][i])
    ax[i].set_xlabel('time [model years]')
    ax[i].legend(frameon=False)
ax[0].text(.95,.05, 'monthly data &\n10 year lowpass', ha='right', transform=ax[0].transAxes, color='grey')

ax[0].set_xlim((95,305))
ax[1].set_xlim((395,605))
plt.savefig(f'{path_results}/paper3/AMOC_low_high_ctrl_rcp')